In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam, LBFGS
from nodag_calm import nocalm
from SCM_data import generate_scm_data
import numpy as np
from numpy.linalg import LinAlgError, inv
from scipy.linalg import sqrtm
import MEC
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [18]:
# True Graph
n = 10000
W_true = np.array([[0, 1, 0], [0, 0, 0], [0, 1, 0]])
Omega_true = np.eye(3)
Theta_true = (np.eye(3) - W_true) @ inv(Omega_true) @ (np.eye(3) - W_true.T)
print("Theta_true\n", Theta_true)
Sigma_true = inv(Theta_true)
print("Sigma_true\n",Sigma_true)


# Data
X, Y, Z, G_true, CPDAG = generate_scm_data(3, n_samples=n)

data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
A_true = (np.eye(3) - W_true) @ inv(sqrtm(Omega_true))
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ Sigma_true @ A_true)
print("A_true = \n",A_true)

Theta_true
 [[ 2. -1.  1.]
 [-1.  1. -1.]
 [ 1. -1.  2.]]
Sigma_true
 [[1. 1. 0.]
 [1. 3. 1.]
 [0. 1. 1.]]
A_true = 
 [[ 1. -1.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  1.]]


In [19]:
A_est, B_est, info = nocalm(
        R_hat, lam=1, beta=10.0, delta=1e-6,
        max_steps=2000, optimizer_type="lbfgs", lr=0.1, history_every=1000,
        dtype=torch.float64
    )

print(A_est)
print(B_est)

[[ 0.01539005  0.00091678 -0.01718084]
 [ 0.00087682 -0.01081938 -0.00922021]
 [-0.00075618  0.00362606 -0.00717282]]
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]


In [20]:
A_init = np.array([[1, 1, 0],[0, 1, 0],[0, 1, 1]])
A_est, B_est, info = nocalm(
        R_hat, lam=1, beta=10.0, delta=1e-6,
        max_steps=2000, optimizer_type="adam", lr=0.1, history_every=1000,
        dtype=torch.float64,A_init = A_init
    )
print(A_est)
print(B_est)

[[ 0.0320391   0.00154878 -0.01517699]
 [ 0.01353939 -0.00429054 -0.03923077]
 [-0.01726469 -0.0379868  -0.0515936 ]]
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]


In [9]:
# init: ground truth

times = 1
for i in range(1, 6):
    true_count = [0] * 6
    for seed in range(times):
        X, Y, Z, G_true, CPDAG = generate_scm_data(i,10000, seed = seed)
        A_true = (np.eye(3) - G_true)
        data = np.array([X, Y, Z]).T
        R_hat = np.cov(data.T)
        #print(data.T@ data / 10000)
        A_est, B_est, info = nocalm(
            R_hat, lam=0.5, beta=10.0, 
            max_steps=2000, optimizer_type="lbfgs", lr=0.1, history_every=1000,
            dtype=torch.float64,A_init = A_true + np.eye(G_true.shape[0])
    )
        
        print("G_est = \n",B_est)
        if MEC.is_in_markov_equiv_class(G_true, B_est): true_count[i-1] += 1
        print("Final Loss = ", info["final_loss"])
    print(f"SCM {i} : {true_count[i-1]/times}")

G_est = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Final Loss =  2.9558088509832383
SCM 1 : 1.0
G_est = 
 [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 0.]]
Final Loss =  4.1827224493009325
SCM 2 : 1.0
G_est = 
 [[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
Final Loss =  4.542162705376069
SCM 3 : 1.0
G_est = 
 [[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
Final Loss =  4.640448577672834
SCM 4 : 0.0
G_est = 
 [[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]
Final Loss =  5.094627360415677
SCM 5 : 0.0


In [7]:
# init: 0 matrix

times = 1
for i in range(1, 6):
    true_count = [0] * 6
    for seed in range(times):
        X, Y, Z, G_true, CPDAG = generate_scm_data(i,10000, seed = seed)
        data = np.array([X, Y, Z]).T
        R_hat = np.cov(data.T)
        #print(data.T@ data / 10000)
        A_est, B_est, info = nocalm(
            R_hat, lam=1, beta=10.0, delta=1e-6,
            max_steps=2000, optimizer_type="lbfgs", lr=0.1, history_every=1000,
            dtype=torch.float64)
    
        
        #print(Omega_est)
        #print(inv(np.eye(3)-W_est.T)@ Omega_est @ inv(np.eye(3)-W_est))
        #print("G_true = \n",G_true)
        #print("G_est = \n",B_est)
        if MEC.is_in_markov_equiv_class(G_true, B_est): true_count[i-1] += 1
        print("Final Loss = ", info["final_loss"])
    print(f"SCM {i} : {true_count[i-1]/times}")

Final Loss =  26.728871246213973
SCM 1 : 1.0
Final Loss =  26.735782114956372
SCM 2 : 0.0
Final Loss =  26.73768297106476
SCM 3 : 0.0
Final Loss =  26.734634151916467
SCM 4 : 0.0
Final Loss =  26.758507556006272
SCM 5 : 0.0


In [8]:
# init: identity matrix

times = 1
for i in range(1, 6):
    true_count = [0] * 6
    for seed in range(times):
        X, Y, Z, G_true, CPDAG = generate_scm_data(i,10000, seed = seed)
        data = np.array([X, Y, Z]).T
        R_hat = np.cov(data.T)
        #print(data.T@ data / 10000)
        A_est, B_est, info = nocalm(
            R_hat, lam=1, beta=10.0, delta=1e-6,
            max_steps=2000, optimizer_type="lbfgs", lr=0.1, history_every=1000,
            dtype=torch.float64, A_init = np.eye(3))
    
        
        #print(Omega_est)
        #print(inv(np.eye(3)-W_est.T)@ Omega_est @ inv(np.eye(3)-W_est))
        #print("G_true = \n",G_true)
        print("G_est = \n",B_est)
        if MEC.is_in_markov_equiv_class(G_true, B_est): true_count[i-1] += 1
        print("Final Loss = ", info["final_loss"])
    print(f"SCM {i} : {true_count[i-1]/times}")

G_est = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Final Loss =  2.956224279594004
SCM 1 : 1.0
G_est = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Final Loss =  3.935837845564218
SCM 2 : 0.0
G_est = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Final Loss =  4.919871918173216
SCM 3 : 0.0
G_est = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Final Loss =  5.880016898753651
SCM 4 : 0.0
G_est = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
Final Loss =  6.907718710390206
SCM 5 : 0.0


In [14]:
# init: random
A_init = np.random.rand(3,3)

times = 1
for i in range(1, 6):
    true_count = [0] * 6
    for seed in range(times):
        X, Y, Z, G_true, CPDAG = generate_scm_data(i,10000, seed = seed)
        data = np.array([X, Y, Z]).T
        R_hat = np.cov(data.T)
        #print(data.T@ data / 10000)
        A_est, B_est, info = nocalm(
            R_hat, lam=0.2, beta=10.0, delta=1e-6,
            max_steps=5000, optimizer_type="lbfgs", lr=0.1, history_every=1000,
            dtype=torch.float64,A_init = A_init)
        
        #print(Omega_est)
        #print(inv(np.eye(3)-W_est.T)@ Omega_est @ inv(np.eye(3)-W_est))
        #print("G_true = \n",G_true)
        print("G_est = \n",B_est)
        if MEC.is_in_markov_equiv_class(G_true, B_est): true_count[i-1] += 1
        print("Final Loss = ", info["final_loss"])
    print(f"SCM {i} : {true_count[i-1]/times}")

G_est = 
 [[0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 0.]]
Final Loss =  7.6075391700848565
SCM 1 : 0.0
G_est = 
 [[-1.  1.  1.]
 [ 1.  0.  1.]
 [ 0.  1.  0.]]
Final Loss =  8.924527783315032
SCM 2 : 0.0
G_est = 
 [[0. 1. 1.]
 [0. 0. 1.]
 [1. 1. 0.]]
Final Loss =  8.869287775387047
SCM 3 : 0.0
G_est = 
 [[0. 1. 0.]
 [0. 0. 1.]
 [1. 1. 0.]]
Final Loss =  8.993236222953119
SCM 4 : 0.0
G_est = 
 [[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
Final Loss =  11.163653030905031
SCM 5 : 0.0


In [3]:
# X -> Y <- Z

i = 3
X, Y, Z, G_true, CPDAG = generate_scm_data(i,10000, seed = 1)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
Omega_true = np.eye(3)
beta = 10
lam = 1

A_true = (np.eye(3) - G_true) @ inv(sqrtm(Omega_true))
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)


true_penalty = lam * np.sum(np.tanh(beta * np.abs(A_true))[~np.eye(A_true.shape[0], dtype=bool)])

f_true = likelihood_true + true_penalty
print("penalty_true = ", true_penalty)
print("f_true = ", f_true)

#print(data.T@ data / 10000)
A_est, B_est, info = nocalm(
    R_hat, lam=lam, beta=beta, 
    max_steps=2000, optimizer_type="lbfgs", lr=0.1, history_every=1000,
    dtype=torch.float64, A_init = np.eye(3))


#print(Omega_est)
#print(inv(np.eye(3)-W_est.T)@ Omega_est @ inv(np.eye(3)-W_est))
#print("G_true = \n",G_true)
print("A_true = \n",A_true)
print("A_est = \n", A_est)
print("G_est = \n", B_est)
likelihood_final = - 2 * np.log(np.linalg.det(A_est)) + np.trace(A_est.T @ R_hat @ A_est)
penalty_final = lam * np.sum(np.tanh(beta * np.abs(A_est))[~np.eye(A_est.shape[0], dtype=bool)])
print("likelihood_final = ",likelihood_final)
print("penalty_final = ",penalty_final)
print("f_final = ", likelihood_final + penalty_final)
print("final loss:", info["final_loss"])
print("final llikelihood:", info["final_llikelihood"])
print("final penalty:", info["final_penalty"])

likelihood_true =  2.9921669186153315
penalty_true =  1.9999999917553855
f_true =  4.992166910370717
A_true = 
 [[ 1. -1.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  1.]]
A_est = 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
G_est = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
likelihood_final =  5.0430173873628155
penalty_final =  0.0
f_final =  5.0430173873628155
final loss: 5.0430173873628155
final llikelihood: 5.0430173873628155
final penalty: 0.0


In [6]:
X, Y, Z, G_true, CPDAG = generate_scm_data(i,10000, seed = 1)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
Omega_true = np.eye(3)
A_true = (np.eye(3) - G_true) @ inv(sqrtm(Omega_true))
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)

lams = np.logspace(np.log10(0.01), np.log10(10), num=10)
betas = np.logspace(np.log10(1), np.log10(100), num=10)

for lam in (lams):
    for beta in (10, 10):
        A_est, B_est, info = nocalm(
        R_hat,
        lam=lam,
        beta=beta,
        max_steps=2000,
        optimizer_type="lbfgs",
        lr=0.03,
        history_every=500,
        dtype=torch.float64,
        A_init = A_true
    )
        if MEC.is_in_markov_equiv_class(G_true, B_est): print(f"lam = {lam}, beta = {beta}")
        print(B_est)

    

lam = 0.01, beta = 10
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
lam = 0.01, beta = 10
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
lam = 0.021544346900318832, beta = 10
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
lam = 0.021544346900318832, beta = 10
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
lam = 0.046415888336127774, beta = 10
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
lam = 0.046415888336127774, beta = 10
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
lam = 0.1, beta = 10
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
lam = 0.1, beta = 10
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
lam = 0.21544346900318834, beta = 10
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
lam = 0.21544346900318834, beta = 10
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
lam = 0.46415888336127775, beta = 10
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
lam = 0.46415888336127775, beta = 10
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
lam = 1.0, beta = 10
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
lam = 1.0, beta = 10
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
lam = 2.154434690031882, beta = 10
[[0. 1. 0.]
 [0. 0.

In [12]:
!pip install optuna


   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import optuna

def objective(trial):
    
    lam = trial.suggest_float("lam", 1e-3, 1.0, log=True)
    beta = trial.suggest_float("beta", 1.0, 50.0)

    A_est, B_est, info = nocalm(
        R_hat,
        lam=lam,
        beta=beta,
        max_steps=2000,
        optimizer_type="lbfgs",
        lr=0.03,
        history_every=500,
        dtype=torch.float64,
    )

    # 最小化最终 loss
    return info["final_loss"]

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Best params:", study.best_params)
print("Best value:", study.best_value)


[I 2025-09-07 17:49:09,731] A new study created in memory with name: no-name-bab97863-32b6-4abc-8365-1b2a24aa5530
[I 2025-09-07 17:49:11,796] Trial 0 finished with value: 3.008435239296924 and parameters: {'lam': 0.003212672273401207, 'beta': 37.974715219027566}. Best is trial 0 with value: 3.008435239296924.
[I 2025-09-07 17:49:13,626] Trial 1 finished with value: 3.079211344880811 and parameters: {'lam': 0.04593955822038763, 'beta': 1.757920248531243}. Best is trial 0 with value: 3.008435239296924.
[I 2025-09-07 17:49:15,444] Trial 2 finished with value: 3.0254851440265407 and parameters: {'lam': 0.008290954626115205, 'beta': 30.8690988869943}. Best is trial 0 with value: 3.008435239296924.
[I 2025-09-07 17:49:17,167] Trial 3 finished with value: 3.018719703718669 and parameters: {'lam': 0.005610110752202096, 'beta': 38.09855734858557}. Best is trial 0 with value: 3.008435239296924.
[I 2025-09-07 17:49:18,983] Trial 4 finished with value: 3.1262959239139643 and parameters: {'lam': 0.

Best params: {'lam': 0.0010933092269752632, 'beta': 1.0292700573889908}
Best value: 2.993800792604806


In [ ]:
#print(data.T@ data / 10000)
lam = 0.0010933092269752632
beta = 1.0292700573889908

print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.sum(np.tanh(beta * np.abs(A_true))[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty
print("penalty_true = ", true_penalty)
print("f_true = ", f_true)

A_est, B_est, info = nocalm(
    R_hat, lam=lam, beta=beta, 
    max_steps=2000, optimizer_type="lbfgs", lr=0.1, history_every=1000,
    dtype=torch.float64, A_init = np.eye(3))


#print(Omega_est)
#print(inv(np.eye(3)-W_est.T)@ Omega_est @ inv(np.eye(3)-W_est))
#print("G_true = \n",G_true)
print("A_true = \n",A_true)
print("A_est = \n", A_est)
print("G_est = \n", B_est)
likelihood_final = - 2 * np.log(np.linalg.det(A_est)) + np.trace(A_est.T @ R_hat @ A_est)
penalty_final = lam * np.sum(np.tanh(beta * np.abs(A_est))[~np.eye(A_est.shape[0], dtype=bool)])
print("likelihood_final = ",likelihood_final)
print("penalty_final = ",penalty_final)
print("f_final = ", likelihood_final + penalty_final)
print("Final Loss = ", info["final_loss"])

likelihood_true =  2.9921669186153315
penalty_true =  0.001691601752228454
f_true =  2.99385852036756
A_true = 
 [[ 1. -1.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  1.]]
A_est = 
 [[ 1.35652315 -0.24724542  0.34572889]
 [-0.48937904  0.72751406 -0.4878873 ]
 [ 0.34712348 -0.24778022  1.35252283]]
G_est = 
 [[0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 0.]]
likelihood_final =  2.991835124341749
penalty_final =  0.002309055115949235
f_final =  2.994144179457698
Final Loss =  2.9927683578636923
